In [1]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import xgboost as xgb
path ='../data/'
train = pd.read_csv(path +'train.csv')
test = pd.read_csv(path +'test.csv')
print (train.shape)

C:\Explorer\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (4,5,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(612652, 27)


C:\Explorer\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
train = train[train.gender != '\\N']
# test = test[test.gender != '\\N']
train['gender'] = train['gender'].apply(lambda x : int(x))
test['gender'] = test['gender'].apply(lambda x : int(x))

In [3]:
train = train[train.age != '\\N']
# test = test[test.age != '\\N']
train['age'] = train['age'].apply(lambda x : int(x))
test['age'] = test['age'].apply(lambda x : int(x))


In [4]:
train = train[train['2_total_fee'] != '\\N']
# test = test[test['2_total_fee'] != '\\N']
test.loc[test['2_total_fee'] == '\\N','2_total_fee'] = 0.0
train['2_total_fee'] = train['2_total_fee'].apply(lambda x : float(x))
test['2_total_fee'] = test['2_total_fee'].apply(lambda x : float(x))

In [5]:
train = train[train['3_total_fee'] != '\\N']
# test = test[test['3_total_fee'] != '\\N']
test.loc[test['3_total_fee'] == '\\N','3_total_fee'] = 0.0
train['3_total_fee'] = train['3_total_fee'].apply(lambda x : float(x))
test['3_total_fee'] = test['3_total_fee'].apply(lambda x : float(x))

In [6]:
label = train.pop('current_service')
le = LabelEncoder()
label = le.fit_transform(label)

In [7]:
feature = [value for value in train.columns.values if
                   value not in ['user_id']]


In [11]:
#xgb模型
def XGB():
    clf = xgb.XGBClassifier(max_depth=12, learning_rate=0.045,
                            n_estimators=752, silent=True,
                            objective="multi:softmax",
                            nthread=4, gamma=0,
                            max_delta_step=0, subsample=1, colsample_bytree=0.9, colsample_bylevel=0.9,
                            reg_alpha=1, reg_lambda=1, scale_pos_weight=1,
                            base_score=0.5, seed=2018, missing=None,num_class=15)
    return clf

In [13]:
online = False
# online = True
if online:
        print ('online')

        model = XGB()
        model.fit(train[feature], label, eval_set=[(train[feature], label)], verbose=1,)
        pred = model.predict(test[feature])
        pred = le.inverse_transform(pred)
        test['predict'] = pred

        test[['user_id', 'predict']].to_csv('./sub.csv', index=False)
else:
        print ('offline')
        train_x,test_x,train_y,test_y = train_test_split(train[feature],label,test_size=0.1,shuffle=True,random_state=2018)
        model = XGB()
        model.fit(train_x[feature], train_y, eval_set=[(test_x[feature], test_y)], verbose=1,early_stopping_rounds=100)
        pred = model.predict(test_x)
        print(f1_score(test_y,pred,average='weighted'))

        # feature_list = model.feature_importances_
        # pd.DataFrame(
        #         {
        #                 'feature':feature,
        #                 'score':feature_list,
        #         }
        # ).to_csv('./feature_importance.csv',index=False)

        # from sklearn.externals import joblib
        # joblib.dump(model, 'gbm.pkl')

offline
[0]	validation_0-merror:0.10293
Will train until validation_0-merror hasn't improved in 100 rounds.
[1]	validation_0-merror:0.096074
[2]	validation_0-merror:0.091945
[3]	validation_0-merror:0.090002
[4]	validation_0-merror:0.089007
[5]	validation_0-merror:0.088011
[6]	validation_0-merror:0.086901
[7]	validation_0-merror:0.08664
[8]	validation_0-merror:0.086232
[9]	validation_0-merror:0.085759
[10]	validation_0-merror:0.085661
[11]	validation_0-merror:0.085073
[12]	validation_0-merror:0.084959
[13]	validation_0-merror:0.084698
[14]	validation_0-merror:0.084436
[15]	validation_0-merror:0.084012
[16]	validation_0-merror:0.083947
[17]	validation_0-merror:0.083914
[18]	validation_0-merror:0.083457
[19]	validation_0-merror:0.083245
[20]	validation_0-merror:0.083033
[21]	validation_0-merror:0.083212
[22]	validation_0-merror:0.083098
[23]	validation_0-merror:0.082576
[24]	validation_0-merror:0.082396
[25]	validation_0-merror:0.082168
[26]	validation_0-merror:0.082004
[27]	validation_0-

[236]	validation_0-merror:0.065437
[237]	validation_0-merror:0.065404
[238]	validation_0-merror:0.065306
[239]	validation_0-merror:0.065404
[240]	validation_0-merror:0.065241
[241]	validation_0-merror:0.065143
[242]	validation_0-merror:0.065176
[243]	validation_0-merror:0.065111
[244]	validation_0-merror:0.065062
[245]	validation_0-merror:0.065062
[246]	validation_0-merror:0.065062
[247]	validation_0-merror:0.065078
[248]	validation_0-merror:0.065029
[249]	validation_0-merror:0.065111
[250]	validation_0-merror:0.06498
[251]	validation_0-merror:0.064915
[252]	validation_0-merror:0.064849
[253]	validation_0-merror:0.064719
[254]	validation_0-merror:0.064768
[255]	validation_0-merror:0.064686
[256]	validation_0-merror:0.064751
[257]	validation_0-merror:0.064703
[258]	validation_0-merror:0.064572
[259]	validation_0-merror:0.06467
[260]	validation_0-merror:0.064507
[261]	validation_0-merror:0.064605
[262]	validation_0-merror:0.064605
[263]	validation_0-merror:0.064605
[264]	validation_0-mer

[471]	validation_0-merror:0.060051
[472]	validation_0-merror:0.059985
[473]	validation_0-merror:0.060002
[474]	validation_0-merror:0.060034
[475]	validation_0-merror:0.060051
[476]	validation_0-merror:0.060018
[477]	validation_0-merror:0.060051
[478]	validation_0-merror:0.060018
[479]	validation_0-merror:0.0601
[480]	validation_0-merror:0.060067
[481]	validation_0-merror:0.0601
[482]	validation_0-merror:0.060067
[483]	validation_0-merror:0.059953
[484]	validation_0-merror:0.059904
[485]	validation_0-merror:0.059887
[486]	validation_0-merror:0.059822
[487]	validation_0-merror:0.059789
[488]	validation_0-merror:0.059757
[489]	validation_0-merror:0.059806
[490]	validation_0-merror:0.059822
[491]	validation_0-merror:0.059838
[492]	validation_0-merror:0.059757
[493]	validation_0-merror:0.059806
[494]	validation_0-merror:0.059838
[495]	validation_0-merror:0.059838
[496]	validation_0-merror:0.059773
[497]	validation_0-merror:0.059773
[498]	validation_0-merror:0.059806
[499]	validation_0-merro

[706]	validation_0-merror:0.058614
[707]	validation_0-merror:0.058582
[708]	validation_0-merror:0.058582
[709]	validation_0-merror:0.058598
[710]	validation_0-merror:0.058663
[711]	validation_0-merror:0.058614
[712]	validation_0-merror:0.058582
[713]	validation_0-merror:0.058631
[714]	validation_0-merror:0.058614
[715]	validation_0-merror:0.058598
[716]	validation_0-merror:0.058565
[717]	validation_0-merror:0.058582
[718]	validation_0-merror:0.058598
[719]	validation_0-merror:0.058549
[720]	validation_0-merror:0.058467
[721]	validation_0-merror:0.0585
[722]	validation_0-merror:0.058451
[723]	validation_0-merror:0.058484
[724]	validation_0-merror:0.058451
[725]	validation_0-merror:0.058484
[726]	validation_0-merror:0.058484
[727]	validation_0-merror:0.0585
[728]	validation_0-merror:0.058402
[729]	validation_0-merror:0.058467
[730]	validation_0-merror:0.058484
[731]	validation_0-merror:0.0585
[732]	validation_0-merror:0.058451
[733]	validation_0-merror:0.0585
[734]	validation_0-merror:0.

C:\Explorer\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [12]:
#online = False
online = True
if online:
        print ('online')

        model = XGB()
        model.fit(train[feature], label, eval_set=[(train[feature], label)], verbose=1,)
        pred = model.predict(test[feature])
        pred = le.inverse_transform(pred)
        test['predict'] = pred

        test[['user_id', 'predict']].to_csv('./sub2.csv', index=False)
else:
        print ('offline')
        train_x,test_x,train_y,test_y = train_test_split(train[feature],label,test_size=0.1,shuffle=True,random_state=2018)
        model = XGB()
        model.fit(train_x[feature], train_y, eval_set=[(test_x[feature], test_y)], verbose=1,early_stopping_rounds=100)
        pred = model.predict(test_x)
        print(f1_score(test_y,pred,average='weighted'))

        # feature_list = model.feature_importances_
        # pd.DataFrame(
        #         {
        #                 'feature':feature,
        #                 'score':feature_list,
        #         }
        # ).to_csv('./feature_importance.csv',index=False)

        # from sklearn.externals import joblib
        # joblib.dump(model, 'gbm.pkl')

online
[0]	validation_0-merror:0.093258
[1]	validation_0-merror:0.085333
[2]	validation_0-merror:0.081879
[3]	validation_0-merror:0.079734
[4]	validation_0-merror:0.078078
[5]	validation_0-merror:0.076798
[6]	validation_0-merror:0.076075
[7]	validation_0-merror:0.075389
[8]	validation_0-merror:0.074328
[9]	validation_0-merror:0.073664
[10]	validation_0-merror:0.073124
[11]	validation_0-merror:0.072631
[12]	validation_0-merror:0.07228
[13]	validation_0-merror:0.071947
[14]	validation_0-merror:0.071668
[15]	validation_0-merror:0.071372
[16]	validation_0-merror:0.070876
[17]	validation_0-merror:0.070437
[18]	validation_0-merror:0.07008
[19]	validation_0-merror:0.069745
[20]	validation_0-merror:0.069456
[21]	validation_0-merror:0.069149
[22]	validation_0-merror:0.068909
[23]	validation_0-merror:0.068586
[24]	validation_0-merror:0.06824
[25]	validation_0-merror:0.067894
[26]	validation_0-merror:0.067608
[27]	validation_0-merror:0.067297
[28]	validation_0-merror:0.067097
[29]	validation_0-me

[238]	validation_0-merror:0.02903
[239]	validation_0-merror:0.028878
[240]	validation_0-merror:0.0288
[241]	validation_0-merror:0.028618
[242]	validation_0-merror:0.028483
[243]	validation_0-merror:0.028377
[244]	validation_0-merror:0.028261
[245]	validation_0-merror:0.028068
[246]	validation_0-merror:0.027961
[247]	validation_0-merror:0.027809
[248]	validation_0-merror:0.027673
[249]	validation_0-merror:0.027567
[250]	validation_0-merror:0.02745
[251]	validation_0-merror:0.027337
[252]	validation_0-merror:0.027259
[253]	validation_0-merror:0.027073
[254]	validation_0-merror:0.027017
[255]	validation_0-merror:0.026875
[256]	validation_0-merror:0.026769
[257]	validation_0-merror:0.026575
[258]	validation_0-merror:0.026495
[259]	validation_0-merror:0.026399
[260]	validation_0-merror:0.026239
[261]	validation_0-merror:0.026201
[262]	validation_0-merror:0.026098
[263]	validation_0-merror:0.025922
[264]	validation_0-merror:0.025826
[265]	validation_0-merror:0.025736
[266]	validation_0-merro

[473]	validation_0-merror:0.008816
[474]	validation_0-merror:0.008767
[475]	validation_0-merror:0.008713
[476]	validation_0-merror:0.008689
[477]	validation_0-merror:0.008643
[478]	validation_0-merror:0.008596
[479]	validation_0-merror:0.008545
[480]	validation_0-merror:0.008517
[481]	validation_0-merror:0.008468
[482]	validation_0-merror:0.008418
[483]	validation_0-merror:0.008403
[484]	validation_0-merror:0.008369
[485]	validation_0-merror:0.008328
[486]	validation_0-merror:0.008289
[487]	validation_0-merror:0.00824
[488]	validation_0-merror:0.008192
[489]	validation_0-merror:0.008158
[490]	validation_0-merror:0.00813
[491]	validation_0-merror:0.00807
[492]	validation_0-merror:0.008003
[493]	validation_0-merror:0.007947
[494]	validation_0-merror:0.007926
[495]	validation_0-merror:0.007915
[496]	validation_0-merror:0.007856
[497]	validation_0-merror:0.007827
[498]	validation_0-merror:0.007791
[499]	validation_0-merror:0.007753
[500]	validation_0-merror:0.007709
[501]	validation_0-merr

[708]	validation_0-merror:0.00261
[709]	validation_0-merror:0.002603
[710]	validation_0-merror:0.002586
[711]	validation_0-merror:0.002566
[712]	validation_0-merror:0.002543
[713]	validation_0-merror:0.002533
[714]	validation_0-merror:0.002522
[715]	validation_0-merror:0.002509
[716]	validation_0-merror:0.002506
[717]	validation_0-merror:0.002491
[718]	validation_0-merror:0.002476
[719]	validation_0-merror:0.002458
[720]	validation_0-merror:0.00244
[721]	validation_0-merror:0.00243
[722]	validation_0-merror:0.002416
[723]	validation_0-merror:0.002403
[724]	validation_0-merror:0.002391
[725]	validation_0-merror:0.002368
[726]	validation_0-merror:0.002347
[727]	validation_0-merror:0.002334
[728]	validation_0-merror:0.002331
[729]	validation_0-merror:0.002323
[730]	validation_0-merror:0.002298
[731]	validation_0-merror:0.00228
[732]	validation_0-merror:0.002274
[733]	validation_0-merror:0.002259
[734]	validation_0-merror:0.002241
[735]	validation_0-merror:0.002223
[736]	validation_0-merro

C:\Explorer\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Explorer\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
